In [25]:
import pandas as pd
import aaindexer



In [26]:
data = pd.read_excel("H:\\Documents\\VUB\\advanced_methods_in_bioInformatics\\BioInformatics\\data_structerd.xlsx")
data = data.sample(n=1000, random_state=42)


In [27]:
print(data.head(5))
type(data["Mutation"])
#type(data["Mutation"][4])


       Unnamed: 0 UniProt ID   Gene Mutation      Class
7308         7308     P12110  CO6A2    R366Q     Driver
635           635     Q96NR3  PTHD1    I139T     Driver
16581       16581     Q15813   TBCE    V158L     Driver
16705       16705     P01266   THYG   R1423G  Passenger
4978         4978     Q01484   ANK2   E2109K  Passenger


pandas.core.series.Series

## Physicochemical properties

In [28]:
n= 10 # number of features to extract fro aindex data base 
aindex_base = aaindexer.scrape_parse(1, progress=False)[:n]

In [29]:
aindex_base[0]

AaindexRecord(accession='ANDN920101', description='alpha-CH chemical shifts (Andersen et al., 1992)', pmid='PMID:1575719', authors='Andersen, N.H., Cao, B. and Chen, C.', title='Peptide/protein structure analysis using the chemical shift index method:  upfield alpha-CH values reveal dynamic helices and aL sites', journal='Biochem. and Biophys. Res. Comm. 184, 1008-1014 (1992)', comment=None, correlation={'BUNA790102': 0.949}, index={'A': 4.35, 'R': 4.38, 'N': 4.75, 'D': 4.76, 'C': 4.65, 'Q': 4.37, 'E': 4.29, 'G': 3.97, 'H': 4.63, 'I': 3.95, 'L': 4.17, 'K': 4.36, 'M': 4.52, 'F': 4.66, 'P': 4.44, 'S': 4.5, 'T': 4.35, 'W': 4.7, 'Y': 4.6, 'V': 3.95}, matrix=None)

In [30]:
for a_row in aindex_base:
    prop = a_row.accession

    # Initialize the new column if not already present
    if prop not in data.columns:
        data[prop] = None

    for index, d_row in data.iterrows():
        try:
            wild_residue = d_row["Mutation"][0]
            mutant_residue = d_row["Mutation"][-1]

            # Ensure both residues are in the index dictionary
            if mutant_residue in a_row.index and wild_residue in a_row.index:
                diff = a_row.index[mutant_residue] - a_row.index[wild_residue]
                data.at[index, prop] = diff
            else:
                data.at[index, prop] = None  # or np.nan
        except Exception as e:
            print(f"Error at row {index}: {e}")
            data.at[index, prop] = None

        
    

        
    

In [31]:
print(data.head(10))


       Unnamed: 0 UniProt ID   Gene Mutation      Class ANDN920101 ARGP820101  \
7308         7308     P12110  CO6A2    R366Q     Driver      -0.01       -0.6   
635           635     Q96NR3  PTHD1    I139T     Driver        0.4      -2.17   
16581       16581     Q15813   TBCE    V158L     Driver       0.22       0.21   
16705       16705     P01266   THYG   R1423G  Passenger      -0.41      -0.53   
4978         4978     Q01484   ANK2   E2109K  Passenger       0.07       0.68   
10174       10174     Q96RW7  HMCN1   R5205L  Passenger      -0.21       0.93   
7882         7882     P11532    DMD   C1892Y  Passenger      -0.05       0.81   
9725         9725     Q6P531   GGT6     G52R     Driver       0.41       0.53   
4100         4100     O95177  GAAS1     V60I     Driver        0.0        0.9   
4201         4201     Q5VV41  ARHGG    A552T     Driver        0.0      -0.56   

      ARGP820102 ARGP820103 BEGF750101 BEGF750102 BEGF750103 BHAR880101  \
7308        0.52       0.06      

## Di- and tri-peptide motifs:

In [32]:
pip install biopython pandas requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [33]:
import pandas as pd
import requests
from Bio import SeqIO
from io import StringIO

In [34]:
data["bf_peptide"]= None 
data["af_peptide"]= None
data["bf_peptide_ratio"]= None
data["af_peptide_ratio"]= None

In [35]:
import pandas as pd
import re
import requests
from collections import defaultdict

# Step 1: Fetch UniProt sequences
def fetch_uniprot_sequence(uniprot_id):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.fasta"
    try:
        r = requests.get(url)
        if r.ok:
            lines = r.text.strip().split('\n')
            return ''.join(lines[1:])  # remove FASTA header
    except Exception as e:
        print(f"Error fetching {uniprot_id}: {e}")
    return None

uniprot_ids = data['UniProt ID'].unique()
uniprot_sequences = {uid: fetch_uniprot_sequence(uid) for uid in uniprot_ids}

def extract_dipeptides(seq, mutation):
    match = re.match(r"([A-Z])(\d+)([A-Z])", mutation)
    if not match:
        return None, None
    _, pos_str, _ = match.groups()
    pos = int(pos_str) - 1
    if pos <= 0 or pos >= len(seq) - 1:
        return None, None
    # Skip AA check: just extract neighbors blindly
    return seq[pos - 1] + seq[pos], seq[pos] + seq[pos + 1]


# Step 3: Count dipeptides by class
dipeptide_counts = defaultdict(lambda: {'Driver': 0, 'Passenger': 0})

for _, row in data.iterrows():
    seq = uniprot_sequences.get(row['UniProt ID'])
    if not seq:
        continue
    before, after = extract_dipeptides(seq, row['Mutation'])
    data.at[row.name, 'bf_peptide'] = before
    data.at[row.name, 'af_peptide'] = after
    for dipep in [before, after]:
        if dipep:
            dipeptide_counts[dipep][row['Class']] += 1


    



# Step 4: Compute odds ratios
results = []

total_driver = sum(c['Driver'] for c in dipeptide_counts.values())
total_Passenger = sum(c['Passenger'] for c in dipeptide_counts.values())

if total_driver == 0 or total_Passenger == 0:
    print("⚠️ Cannot compute odds ratios: one of the classes has zero total.")
else:
    for dipep, counts in dipeptide_counts.items():
        ndp = counts['Driver']
        nnp = counts['Passenger']
        odds = (ndp / total_driver) / (nnp / total_Passenger) if nnp > 0 else float('inf')

        if odds >= 1.2:
            label =  "i"  #"highly_preferred"
        elif odds <= 0.8:
            label = "ii" #"not_preferred"
        else:
            label = "c"  #"Moderate"

        results.append((dipep, ndp, nnp, round(odds, 2), label))

# Step 5: Display results
odds_df = pd.DataFrame(results, columns=["Dipeptide", "Driver_count", "Passenger_count", "Odds_ratio", "Class"])
print(odds_df)


    Dipeptide  Driver_count  Passenger_count  Odds_ratio Class
0          ER             9                8        1.06     c
1          RG             8                3        2.52     i
2          HI             3                3        0.94     c
3          IC             1                0         inf     i
4          KV             4                4        0.94     c
..        ...           ...              ...         ...   ...
347        TF             1                0         inf     i
348        FY             1                0         inf     i
349        EY             2                0         inf     i
350        FL             1                1        0.94     c
351        YP             0                1        0.00    ii

[352 rows x 5 columns]


In [36]:
highly_preferred_dipeptides = odds_df[odds_df['Odds_ratio'] == float('inf')]
print("Highly preferred dipeptides (Odds ratio = ∞):")
print(highly_preferred_dipeptides[['Dipeptide', 'Driver_count', 'Passenger_count']])
# Save the results to csv:
highly_preferred_dipeptides.to_csv("highly_preferred_dipeptides.csv", index=False)

Highly preferred dipeptides (Odds ratio = ∞):
    Dipeptide  Driver_count  Passenger_count
3          IC             1                0
69         FG             3                0
110        ML             4                0
149        MM             1                0
154        GQ             2                0
171        DN             2                0
172        NY             2                0
175        CE             1                0
187        HG             3                0
201        HQ             1                0
219        RY             4                0
221        GF             5                0
222        HR             1                0
226        LW             3                0
232        PH             1                0
235        WR             2                0
238        IK             2                0
239        EN             2                0
240        DC             2                0
261        CD             2                0
264      

In [37]:
passenger_preferred_dipeptides = odds_df[odds_df['Odds_ratio'] == 0.0]
print("Passenger preferred dipeptides (Odds ratio = 0.0):")
print(passenger_preferred_dipeptides[['Dipeptide', 'Driver_count', 'Passenger_count']])
# Save the results to csv:
passenger_preferred_dipeptides.to_csv("passenger_preferred_dipeptides.csv", index=False)


Passenger preferred dipeptides (Odds ratio = 0.0):
    Dipeptide  Driver_count  Passenger_count
26         CS             0                2
33         NI             0                1
35         TY             0                2
49         SI             0                6
50         IM             0                2
82         AN             0                3
90         HT             0                3
136        YN             0                1
141        CG             0                2
167        WC             0                1
169        PC             0                2
170        MD             0                3
179        KS             0                4
210        TW             0                1
211        WS             0                1
231        WK             0                1
247        GH             0                4
262        SY             0                4
267        FE             0                1
279        QK             0                3
281 

In [38]:

for index, row in data.iterrows():
    bf_peptide = row['bf_peptide']
    af_peptide = row['af_peptide']
    
    if bf_peptide : 
        ratio_class= odds_df[odds_df["Dipeptide"] == bf_peptide]["Class"]
        if not ratio_class.empty:
            data.at[index, "bf_peptide_ratio"] = ratio_class.values[0]
    if af_peptide:
        ratio_class= odds_df[odds_df["Dipeptide"] == af_peptide]["Class"]
        if not ratio_class.empty:
            data.at[index, "af_peptide_ratio"] = ratio_class.values[0]

In [39]:

for index, row in data.iterrows():
    bf_peptide = row['bf_peptide']
    af_peptide = row['af_peptide']

    if bf_peptide:
        ratio_class = odds_df[odds_df["Dipeptide"] == bf_peptide]["Odds_ratio"]
        if not ratio_class.empty:
            data.at[index, "bf_peptide_ratio"] = ratio_class.values[0]
    if af_peptide:
        ratio_class = odds_df[odds_df["Dipeptide"] == af_peptide]["Odds_ratio"]
        if not ratio_class.empty:
            data.at[index, "af_peptide_ratio"] = ratio_class.values[0]

In [40]:
data.head(10)

,Unnamed: 0,UniProt ID,Gene,Mutation,Class,ANDN920101,ARGP820101,ARGP820102,ARGP820103,BEGF750101,BEGF750102,BEGF750103,BHAR880101,BIGC670101,BIOV880101,bf_peptide,af_peptide,bf_peptide_ratio,af_peptide_ratio
7308,7308,P12110,CO6A2,R366Q,Driver,-0.01,-0.6,0.52,0.06,-0.08,0.0,-0.2,-0.036,-19.4,-3.0,ER,RG,1.06,2.52
635,635,Q96NR3,PTHD1,I139T,Driver,0.4,-2.17,-0.61,-0.76,-0.29,-0.15,0.38,-0.018,-30.8,-189.0,HI,IC,0.94,inf
16581,16581,Q15813,TBCE,V158L,Driver,0.22,0.21,2.15,1.79,0.18,-0.15,0.16,-0.021,16.9,22.0,KV,VD,0.94,0.94
16705,16705,P01266,THYG,R1423G,Passenger,-0.41,-0.53,0.29,0.17,-0.17,-0.07,0.13,0.015,-72.8,57.0,IR,RF,5.19,1.89
4978,4978,Q01484,ANK2,E2109K,Passenger,0.07,0.68,-0.05,-0.08,-0.13,0.0,0.22,-0.031,20.4,-35.0,SE,EV,0.27,2.36
10174,10174,Q96RW7,HMCN1,R5205L,Passenger,-0.21,0.93,3.03,2.48,0.48,0.11,-0.31,-0.164,-7.1,215.0,QR,RC,0.75,0.27
7882,7882,P11532,DMD,C1892Y,Passenger,-0.05,0.81,-1.5,-0.55,0.38,0.18,0.0,0.074,47.9,-115.0,KC,CL,0.94,0.31
9725,9725,Q6P531,GGT6,G52R,Driver,0.41,0.53,-0.29,-0.17,0.17,0.07,-0.13,-0.015,72.8,-57.0,GG,GL,2.12,4.25
4100,4100,O95177,GAAS1,V60I,Driver,0.0,0.9,0.37,0.53,-0.09,0.0,0.0,0.076,16.9,28.0,PI,IA,6.61,0.38
4201,4201,Q5VV41,ARHGG,A552T,Driver,0.0,-0.56,-0.34,-0.65,-0.56,0.06,0.38,0.087,18.6,-54.0,YA,AQ,1.89,0.38


In [41]:
print(data[data["Class"] == "Driver"].shape)
print(data.shape)


(515, 19)
(1000, 19)


In [49]:
data = data.drop(columns=["Unnamed: 0"])

KeyError: "['Unnamed: 0'] not found in axis"

In [50]:
data.to_csv("features_data_1.csv", index=False)

PermissionError: [Errno 13] Permission denied: 'features_data_1.csv'

In [51]:
# have the final _data

data_2= pd.read_csv("faraah_my_final_merged_featuress.csv")

final_data = pd.merge(data, data_2, on=["Mutation","UniProt ID" ], how="inner")
final_data.dropna(inplace=True)
final_data.to_csv("final_data.csv", index=False)
final_data.shape

(995, 251)